# Importe

In [5]:
# general
import pandas as pd

# waits
import random
from time import sleep

# driver
import undetected_chromedriver.v2 as uc
from selenium import webdriver
import chromedriver_autoinstaller

# selenium
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# other
# from ssl import Options

# multi
from concurrent import futures


# Driver

In [6]:
def get_driver(ip="", port="", flag_proxy=False):
    # PATH = "C:\Program Files (x86)\chromedriver.exe"
    chrome_options = uc.ChromeOptions()

    # Removes navigator.webdriver flag
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")

    # prevent bugs and skip popups
    chrome_options.add_argument('--no-first-run --no-service-autorun --password-store=basic')
    chrome_options.add_argument("--disable-extensions")

    # uc - user agent
    # bot detection can determine if os in user-agent differs from real os
    # chrome_options.add_argument("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.5 Safari/605.1.15")
    # chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36")

    # add multiple download flag
    dl_multi = {'profile.default_content_setting_values.automatic_downloads': 1}
    chrome_options.add_experimental_option("prefs", dl_multi)

    # add download location
    dl_path = {"download.default_directory" : "C:\\Users\\lange\\OneDrive\\Dokumente\\Studium\\06_Abschlussarbeiten\\Seminar\\02_Download"}
    chrome_options.add_experimental_option("prefs", dl_path)

    # add profile
    # uc.Chrome(user_data_dir='C:\\Users\\lange\\AppData\\Local\\Google\\Chrome\\User Data\\Default')
    
    # add proxy if True
    # -1 to use first element after use of driver to scrape proxies
    if flag_proxy == True:       
        PROXY = ip + ":" + port
        print(PROXY)
        chrome_options.add_argument('--proxy-server=http://%s' % PROXY)

    # be careful with headless browsing
    # chrome_options.add_argument("--headless")

    # webdriver vs. undetected chromedriver
    # driver = webdriver.Chrome(executable_path=PATH, options=chrome_options)
    driver = uc.Chrome(options=chrome_options)
    driver.maximize_window()
    
    return driver

# Proxies

In [7]:
# chromedriver_autoinstaller.install() # To update your chromedriver automatically
# driver = webdriver.Chrome("C:\\Program Files (x86)\\chromedriver")

# Get free proxies for rotating
def get_list_proxies(driver):
    driver.get('https://sslproxies.org')

    table = driver.find_element(By.TAG_NAME, 'table')
    thead = table.find_element(By.TAG_NAME, 'thead').find_elements(By.TAG_NAME, 'th')
    tbody = table.find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr')

    headers = []
    for th in thead:
        headers.append(th.text.strip())

    proxies = []
    for tr in tbody:
        proxy_data = {}
        tds = tr.find_elements(By.TAG_NAME, 'td')
        for i in range(len(headers)):
            proxy_data[headers[i]] = tds[i].text.strip()
        proxies.append(proxy_data)
    
    driver.quit()
    return proxies

# Scraper

In [18]:
def scrape(swort, nsitz, ip, port, click_count, select_ip, valid_proxy, flag_proxy=False):
    print(swort)
    print(nsitz)
    
    try:
        driver = get_driver(flag_proxy=flag_proxy, ip=ip, port=port)
        URL = "https://www.handelsregister.de/rp_web/erweitertesuche.xhtml"
        # URL = "https://bot.sannysoft.com/"    # test site for bot detection
        # URL = "https://nowsecure.nl"          # test site with max anti-bot protection
        driver.get(URL)
        sleep(10) 
        
        # input
        input_swort = WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.ID, "form:schlagwoerter")))
        input_nsitz = WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.ID, "form:NiederlassungSitz")))
        input_swort.clear()
        input_nsitz.clear()

        input_swort.send_keys(swort)
        sleep(round(random.uniform(0.31, 2.11), 2))
        input_nsitz.send_keys(nsitz)
        sleep(round(random.uniform(0.31, 2.11), 2))

        # submit does not work, so click button
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(round(random.uniform(0.31, 2.11), 2))
        # btn_suche = driver.find_element_by_xpath(xpath="//*[contains(text(),'Suchen')]")
        driver.find_element(By.ID, "form:btnSuche").click()
        sleep(round(random.uniform(0.31, 2.11), 2))
        # extract and click clickable elements
        elements = WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.LINK_TEXT, "AD")))
        sleep(round(random.uniform(0.31, 2.11), 2))
        
    except Exception as exc:
        select_ip +=1
        valid_proxy = False
        print('ERROR with ip: {0} and port: {1} generated an exception: {2}\n try next ...'.format(ip, port, exc))
        driver.quit()
        return click_count, select_ip, valid_proxy
     
    index_elements = len(elements) - 1
    q = 0
    while q <= index_elements:
        try:
            elements[q].click()
            sleep(round(random.uniform(0.31, 2.11), 2))
            # download and go back
            driver.find_element(By.ID, "form:kostenpflichtigabrufen").click()
            click_count += 1
            driver.back()
            print("sucessfully downloaded {0} at index {1}".format(elements[q], q))
        except Exception as exc:
           print('ERROR with swort: {0} , nsitz: {1} and q = {2} generated an exception: {3}'.format(swort, nsitz, q, exc))
        q += 1    
    else:
        driver.quit()
    
    valid_proxy = True    
    return click_count, select_ip, valid_proxy

# Calls

In [9]:
# call proxy scraper
driver = get_driver()
list_proxies = get_list_proxies(driver)
list_proxies

[{'IP Address': '151.181.91.10',
  'Port': '80',
  'Code': 'US',
  'Country': 'United States',
  'Anonymity': 'elite proxy',
  'Google': 'yes',
  'Https': 'yes',
  'Last Checked': '18 secs ago'},
 {'IP Address': '168.91.234.183',
  'Port': '80',
  'Code': 'US',
  'Country': 'United States',
  'Anonymity': 'anonymous',
  'Google': 'no',
  'Https': 'yes',
  'Last Checked': '10 mins ago'},
 {'IP Address': '213.241.205.1',
  'Port': '8080',
  'Code': 'RU',
  'Country': 'Russian Federation',
  'Anonymity': 'anonymous',
  'Google': 'yes',
  'Https': 'yes',
  'Last Checked': '10 mins ago'},
 {'IP Address': '213.241.205.2',
  'Port': '8080',
  'Code': 'RU',
  'Country': 'Russian Federation',
  'Anonymity': 'anonymous',
  'Google': 'yes',
  'Https': 'yes',
  'Last Checked': '10 mins ago'},
 {'IP Address': '49.0.2.242',
  'Port': '8090',
  'Code': 'ID',
  'Country': 'Indonesia',
  'Anonymity': 'elite proxy',
  'Google': 'no',
  'Https': 'yes',
  'Last Checked': '10 mins ago'},
 {'IP Address': '1

In [10]:
# example df
data = [['INTENSE AG', 'Würzburg'], ['snapADDY GmbH', 'Würzburg']]
df_terms = pd.DataFrame(data, columns=['swort', 'nsitz'])

df_terms


,swort,nsitz
0,INTENSE AG,Würzburg
1,snapADDY GmbH,Würzburg


In [19]:
# call scraper
# check different methods for faster applying functions to data
# https://www.ml4devs.com/articles/pandas-dataframe-apply-function-iterate-over-rows/

select_ip = 0
click_count = 0
for index, row in df_terms.iterrows():
    
    valid_proxy = False
    while valid_proxy == False:
        
        if click_count >= 55:
            select_ip +=1
            click_count = 0
    
        dict_proxy = list_proxies[select_ip]
        ip = dict_proxy["IP Address"]
        port = dict_proxy["Port"]
    
        try:
            click_count, select_ip, valid_proxy = scrape(swort=row['swort'], 
                                                        nsitz=row['nsitz'], 
                                                        flag_proxy=True, 
                                                        ip=ip, 
                                                        port=port, 
                                                        click_count=click_count, 
                                                        select_ip=select_ip, 
                                                        valid_proxy=valid_proxy)
        except Exception as exc:
            print("ERROR {}".format(exc))

INTENSE AG
Würzburg
151.181.91.10:80
ERROR with ip: 151.181.91.10 and port: 80 generated an exception: Message: 

 try next ...
INTENSE AG
Würzburg
168.91.234.183:80
ERROR with ip: 168.91.234.183 and port: 80 generated an exception: Message: unknown error: net::ERR_CONNECTION_RESET
  (Session info: chrome=108.0.5359.22)
Stacktrace:
Backtrace:
	Ordinal0 [0x0049ACD3+2075859]
	Ordinal0 [0x0042EE61+1633889]
	Ordinal0 [0x0032B7BD+571325]
	Ordinal0 [0x00327CD2+556242]
	Ordinal0 [0x0031EC5D+519261]
	Ordinal0 [0x0031FAEC+522988]
	Ordinal0 [0x0031EF27+519975]
	Ordinal0 [0x0031E2EB+516843]
	Ordinal0 [0x0031E231+516657]
	Ordinal0 [0x0031D0A3+512163]
	Ordinal0 [0x0031D5AD+513453]
	Ordinal0 [0x0032D0CE+577742]
	Ordinal0 [0x0038BC7D+965757]
	Ordinal0 [0x0037731C+881436]
	Ordinal0 [0x0038B56A+963946]
	Ordinal0 [0x00377136+880950]
	Ordinal0 [0x0034FEFD+720637]
	Ordinal0 [0x00350F3F+724799]
	GetHandleVerifier [0x0074EED2+2769538]
	GetHandleVerifier [0x00740D95+2711877]
	GetHandleVerifier [0x0052A03A+52

# Other

In [ ]:
# MULTI Threading
id, swort, nsitz = get_terms(terms=terms)
# default number of threads is optimized for cpu cores 
# possible to set with `max_workers` like `futures.ThreadPoolExecutor(max_workers=...)`
with futures.ThreadPoolExecutor() as executor:     
    # store the id for each thread as a dict, so we can know which thread fails
    future_results = { id : executor.submit(scrape(swort, nsitz), terms) for id in terms }
    for id, future in future_results.items(): 
        try:        
           future.result() # can use `timeout` to wait max seconds for each thread  
        except Exception as exc: # can give a exception in some thread
           print('id {0} generated an exception: {1}'.format(id, exc))

In [ ]:
# PROXY TEST
PROXY = "174.138.16.96:8888" # IP:PORT or HOST:PORT

chrome_options = uc.ChromeOptions()
chrome_options.add_argument('--proxy-server=http://%s' % PROXY)

driver = uc.Chrome(options=chrome_options)
driver.get("http://whatismyipaddress.com")